<a href="https://colab.research.google.com/github/edwardptera/Gravimetro/blob/main/RJMCMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from IPython.display import display, display_svg
from IPython.display import Math, Latex
from IPython.display import IFrame, HTML

import time
from datetime import datetime as dt
from pytz import timezone
import random, string

In [ ]:
prefix = 'Parallel'
configdict = {
    'Serialpc': "DISABLE_JIT: 1",
    'Parallel': "DISABLE_JIT: 0\n"
                "NUM_THREADS: 2\n"
                "THREADING_LAYER: 'tbb'"
}

dir0 = '/content/drive/My Drive/Posgrado/Geophysics/Colab/RJMCMC'
name = prefix +\
       dt.now(timezone('America/Bogota')).strftime('_%y%m%d-%H%M_') +\
       ''.join(random.sample(string.hexdigits, 4)
       )
workdir = os.path.join(dir0, name)
os.mkdir(workdir)
os.chdir(workdir)
print('Working Directory:', os.getcwd(), sep='\n')
with open('.numba_config.yaml','w') as cf:
  cf.write(configdict[prefix])
  print('Numba Config:', configdict[prefix], sep='\n', end='\n\n')

In [ ]:
# https://stackoverflow.com/a/57883792
# https://stackoverflow.com/a/57113015
%%capture cap --no-stderr
%%bash
pip install tbb
pip install setuptools
# https://github.com/lycantropos/bentley_ottmann
# http://geomalgorithms.com/a09-_intersect-3.html#Shamos-Hoey-Algorithm
# http://euro.ecom.cmu.edu/people/faculty/mshamos/1976GeometricIntersection.pdf
pip install bentley_ottmann
# https://matplotlib-axes-aligner.readthedocs.io/en/latest/
pip install mpl-axes-aligner
pip install celluloid
pip install watermark
pip install tqdm --upgrade
# https://stackoverflow.com/a/57272111
pip install --upgrade plotly
wget -q https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
chmod +x /usr/local/bin/orca
apt-get install xvfb libgtk2.0-0 libgconf-2-4
pip install -U kaleido

In [ ]:
with open('pip_installs.txt', 'w') as f:
    f.write(cap.stdout)
!pip freeze > requirements.txt

In [ ]:
import pandas as pd
import numpy as np
from numba import njit, prange, config
from numba.np.extensions import cross2d

if config.DISABLE_JIT==1:
  cross2d = np.cross
  prange = range
  print('JIT DISABLED!')
else:
  print('JIT!')

In [ ]:
from IPython.display import set_matplotlib_formats

from numpy.linalg import norm
from numpy import random as rg
from scipy.interpolate import interp1d

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.patches as patches
set_matplotlib_formats('pdf', 'svg')
from matplotlib import style
#style.use('classic')


from collections import Counter
from matplotlib.lines import Line2D
from matplotlib.collections import LineCollection
from matplotlib import colors as pltcolors
from matplotlib.colors import ListedColormap, BoundaryNorm
# https://matplotlib-axes-aligner.readthedocs.io/en/latest/
from mpl_axes_aligner import align


from tqdm.auto import trange, tqdm
from pprint import pprint

from sympy import Point, Polygon
from ground.base import get_context
from bentley_ottmann.planar import contour_self_intersects
context  = get_context()
bPoint   = context.point_cls
bSegment = context.segment_cls
bContour = context.contour_cls

from celluloid import Camera
from matplotlib import animation, rc
from base64 import b64encode

In [ ]:
#https://github.com/rasbt/watermark
%reload_ext watermark
%watermark -v -iv -m

In [ ]:
%%bash
lscpu
nvidia-smi

In [ ]:
def showvideo(filename):
  #https://stackoverflow.com/a/57378660
  mp4 = open(filename, 'rb').read()
  data_url='data:video/mp4;base64,' + b64encode(mp4).decode()
  return display(HTML("""
  <video width=800 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url
  ))

In [ ]:
@njit(
  fastmath=True,
  error_model='numpy',
  )
def inner_angle(Model, i):
  k = len(Model)
  prev = Model[(i-1)%k]
  next = Model[(i+1)%k]
  here = Model[i%k]
  vetcs = prev-here, next-here
  #https://stackoverflow.com/a/14067171
  return -np.arctan2(
      cross2d(*vetcs),
      np.dot(*vetcs)
      )

@njit(
  fastmath=True,
  error_model='numpy',
  )
def grav(Model, XZ):
  k = len(Model)
  lenXZ = len(XZ)
  xietalist = np.empty(lenXZ)
  xietalist = [Model-XZ[i] for i in prange(lenXZ)]
  lenxieta = len(xietalist)
  grav = np.empty(lenxieta)
  for j in prange(lenxieta):
    xi = xietalist[j].T[0]
    eta = xietalist[j].T[1]
    sum = 0
    for i in prange(k):
      A = (xi[i-1]*eta[i] - xi[i]*eta[i-1])/\
          ((xi[i]-xi[i-1])**2 + (eta[i]-eta[i-1])**2)
      B1 = 0.5*(eta[i] - eta[i-1])*\
           np.log((xi[i]**2 + eta[i]**2)/\
          (xi[i-1]**2 + eta[i-1]**2))
      B2 = (xi[i] - xi[i-1])*\
           (np.arctan(xi[i]/eta[i])-\
           np.arctan(xi[i-1]/eta[i-1]))
      sum += A*(B1+B2)
    grav[j] = sum
  return grav

class Model(np.ndarray):
  def __new__(cls, input_array,*args,**kargs):
    return np.asarray(input_array).astype(np.float).view(cls)
  def __str__(self):
    return ','.join(map(str,self.flatten()))
  def area(self):
    # https://stackoverflow.com/a/30408825
    (x,z) = self.T
    return -0.5*(np.dot(x,np.roll(z,1))-np.dot(np.roll(x,1),z))
  def Cgeom(self):
    return np.array(np.mean(self,axis=0))
  def Cmass(self):
    # https://en.wikipedia.org/wiki/Centroid#Of_a_polygon
    k = self.__len__()
    A = self.area()
    (x,z) = self.T
    Cx = np.array(
        [(x[i%k] + x[(i+1)%k])*\
         (x[i%k]*z[(i+1)%k] - x[(i+1)%k]*z[i%k])\
         for i in range(k)]).sum()/A/6
    Cz = np.array(
        [(z[i%k] + z[(i+1)%k])*\
         (x[i%k]*z[(i+1)%k] - x[(i+1)%k]*z[i%k])\
         for i in range(k)]).sum()/A/6
    return np.array([Cx, Cz])
  def move(self, i, r, th):
    k = self.__len__()
    new = self.copy()
    new[i%k] = self[i%k] + r*np.array([np.cos(th), np.sin(th)])
    return new
  def birth(self, i, r, th):
    k = self.__len__()
    p = (self[i%k]+self[(i+1)%k])/2
    return np.insert(self, (i+1)%k, p, axis=0).move(i+1, r, th)
  def death(self, i):
    k = self.__len__()
    return np.delete(self, i%k, axis=0)
  def dvector(self, i):
    k = self.__len__()
    return np.array(
        (self[(i+1)%k] + self[(i-1)%k])/2-
        self[i%k]
        )
  def dradius(self,i):
    return norm(self.dvector(i))
  def dcat(self,i):
    k = self.__len__()
    return norm(self[(i+1)%k] - self[(i-1)%k])/2
  def bcat(self,i):
    k = self.__len__()
    return norm(self[(i+1)%k] - self[i%k])/2
  def vectors(self, i):
    k = self.__len__()
    prev = self[(i-1)%k]
    next = self[(i+1)%k]
    here = self[i%k]
    return np.array([prev-here, next-here])
  @njit(
    fastmath=True,
    error_model='numpy',
    )
  def angle(self, i):
    return inner_angle(self, i)
  def dists(self, i):
    mapvects = map(norm, self.vectors(i))
    return np.fromiter(mapvects, dtype=np.float)
  @njit(
    fastmath=True,
    error_model='numpy',
    )
  def gravitational(self, XZ):
    return grav(self, XZ)
  def notintersect(self):
  # https://github.com/lycantropos/bentley_ottmann 
  # Shamos-Huey algorithms 
   return not contour_self_intersects(
      bContour(list(map(lambda _: bPoint(*_), self))
      ))
   
def RegularModel(p, r, phi=np.pi/2, n=3):
  ths = phi+np.linspace(
      0, -2*np.pi, n,
      endpoint=False
      )
  return Model(
      [p + r*np.array([np.cos(th), np.sin(th)]) for th in ths])
   
def str2Model(str):
  return Model(eval(str)).reshape(-1, 2)

def Model_from_file(filename):
  with open(filename, 'r') as file:
    l = list(map(str2Model, file.readlines()))
    if len(l)==1:
      return l[0]
    else:
      return l
    file.close()

def Model_from_GeoGebra(filename):
  return Model(np.genfromtxt(filename))

def xz_iterp(XZ, start, stop, num=1000+1, kind='cubic'):
  xx = np.linspace(start, stop, num)
  zz = interp1d(*XZ.T, kind='cubic')(xx)
  return np.column_stack((xx, zz))

In [ ]:
@njit(
    fastmath=True,
    error_model='numpy',
    )
def gaussian(x, mu, sigma):
  a = np.sqrt(2*np.pi)*sigma
  b = (x-mu)/sigma
  return np.exp(-0.5*b**2)/a

@njit(
    fastmath=True,
    error_model='numpy',
    )
def birth_death(newModel, oldModel, sigma_a, ps, r, ang=np.pi):
  oldk = len(oldModel)
  newk = len(newModel)
  b = ps[oldk-1, 1]
  d = ps[newk-1, 2]
  a = ang/gaussian(r, 0, sigma_a)
  return a*r*(d/b)/(newk/oldk)

@njit(
    fastmath=True,
    error_model='numpy',
    #parallel=True
    )
def Likelihood(TestModel, XZ, g, sigma=0.2):
  v=g-grav(TestModel, XZ)
  test=np.dot(v,v)/sigma**2
  return np.exp(-0.5*test)

@njit(
    fastmath=True,
    error_model='numpy',
    #parallel=True
    )
def Prior_density(TestModel,gamma=1.6):
  k = len(TestModel)
  wk = (k-2)*np.pi/k
  s = np.array([(inner_angle(TestModel,i)-wk)**2\
              for i in range(k)]).sum()
  test = k**gamma + s/k
  return np.exp(-test)

@njit(
    fastmath=True,
    error_model='numpy',
    #parallel=True
    )
def Relatives(newModel, oldModel, XZ, g, sigma=0.2, gamma=1.6):
  oldk  = len(oldModel)
  newk  = len(newModel)
  oldwk = (oldk-2)*np.pi/oldk
  newwk = (newk-2)*np.pi/newk
  oldv  = g-grav(oldModel,XZ)
  newv  = g-grav(newModel,XZ)
  olds  = np.array([(inner_angle(oldModel,i) - oldwk)**2\
                 for i in range(oldk)]).sum()
  news  = np.array([(inner_angle(newModel,i) - newwk)**2\
                 for i in range(newk)]).sum()
  oldPrior  = oldk**gamma+olds/oldk
  newPrior  = newk**gamma+news/newk
  Prior_rel = newPrior-oldPrior
  oldLikelihood  = np.dot(oldv,oldv)/sigma**2
  newLikelihood  = np.dot(newv,newv)/sigma**2
  Likelihood_Rel = newLikelihood-oldLikelihood
  return np.exp(-(
      Likelihood_Rel/2+
      Prior_rel
      ))

In [ ]:
def acceptanceTest(oldModel, oldi, XZ, g, ps, sigma=0.2, gamma=1.6, ang=np.pi):
  k = len(oldModel)
  moves_arr = ['p', 'b', 'd']
  move = np.random.choice(moves_arr, p=ps[k-1])
  i = rg.choice(
        np.delete(np.arange(k), oldi%k)
        )
  if move==moves_arr[0]:
    sigma_a = min(*oldModel.dists(i))/4
    r  = rg.normal(0,sigma_a)
    th = rg.uniform(0,ang)
    newModel = oldModel.move(i,r,th)
    alpha = Relatives(
        newModel, oldModel, XZ, g,
        sigma=sigma, gamma=gamma
        )
  elif move==moves_arr[1]:
    sigma_a = oldModel.bcat(i)/4
    r  = rg.normal(0,sigma_a)
    th = rg.uniform(0,ang)
    newModel = oldModel.birth(i, r, th)
    alpha = Relatives(
        newModel, oldModel, XZ, g,
        sigma=sigma, gamma=gamma
        )*birth_death(newModel, oldModel, 
                      sigma_a, ps, r, ang=ang)
  elif move==moves_arr[2]:
    sigma_a = oldModel.dcat(i)/4
    r = oldModel.dradius(i)
    newModel = oldModel.death(i)
    alpha = Relatives(
        newModel, oldModel, XZ, g,
        sigma=sigma, gamma=gamma
        )/birth_death(oldModel,newModel,
                      sigma_a, ps, r, ang=ang)
  return (alpha, newModel, i, move)

In [ ]:
%%time

N = 5*10**3
ang = np.pi
l = 1.5
oldModel = Model(
    [(0,0),
     (l,0),
     (l+1/np.sqrt(2), -np.sqrt(2)/2)
     ])
moves = np.empty((N, 2))

i = 1
sigma_a = min(*oldModel.dists(1))/4
for m in range(N):
  (r,th) = (rg.normal(0,sigma_a), rg.uniform(0, ang))
  newModel = oldModel.move(i, r, th)
  moves[m] = newModel[i]

fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.plot(
    *newModel.T,
    marker='o',
    color='darkred',
    linestyle='--'
    )
ax.plot(
    *oldModel.T,
    marker='o',
    color='k',  
    )
ax.scatter(
    *moves.T,
    marker='.',
    color='g',
    alpha=.25
    )
fig.patch.set_visible(False)
ax.axis('off')
plt.savefig('moves.pdf',
            bbox_inches='tight',
            pad_inches=0
            )
plt.show()

In [ ]:
%%time

N = 5*10**3
ang = np.pi
l = 1.5
oldModel = Model(
    [(0,0),
     (l,0),
     (l+1/np.sqrt(2), -np.sqrt(2)/2)
     ])
moves = np.empty((N, 2))

i=0
sigma_a = oldModel.bcat(i)/4
for m in range(N):
  (r,th) = (rg.normal(0,sigma_a), rg.uniform(0, ang))
  newModel = oldModel.birth(i, r, th)
  moves[m] = newModel[i+1]

fig,ax=plt.subplots()
ax.set_aspect('equal')
ax.plot(
    *newModel[:-1].T,
    marker='o',
    color='darkred',
    linestyle='--',
    )
ax.plot(
    *oldModel.birth(i, 0, 0).T,
    marker='o',
    color='k',
    )
ax.scatter(
    *moves.T,
    marker='.',
    color='g',
    alpha=.25
    )
fig.patch.set_visible(False)
ax.axis('off')
plt.savefig('births.pdf',
            bbox_inches='tight',
            pad_inches=0
            )
plt.show()

In [ ]:
%%time

N = 5*10**3
ang=np.pi
l = 1.5
oldModel=Model(
    [(0,0),
     (l,0),
     (l+1/np.sqrt(2), -np.sqrt(2)/2)
     ])
moves = np.empty((N, 2))

i = 1
sigma_a = oldModel.dcat(i)/4
r = oldModel.dradius(i)
newModel = oldModel.death(i)

fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.arrow(
    *oldModel[i],
    *oldModel.dvector(i)*0.9,
    head_width=.05,
    head_length=.05,
    fc='gray', ec='gray'
    )
ax.plot(
    *newModel.T,
    marker='o',
    color='darkred',
    linestyle='--'
    )
ax.plot(
    *oldModel.T,
    marker='o',
    color='k',
    )
fig.patch.set_visible(False)
ax.axis('off')
plt.savefig('death.pdf',
            bbox_inches='tight',
            pad_inches=0
            )
plt.show()

In [ ]:
%%file ../TrueModel.csv
110.0,-20.0, 220.0,-10.0, 290.0,-20.0, 390.0,-20.0, 310.0,-30.0, 250.0,-40.0, 220.0,-60.0, 180.0,-60.0, 170.0,-40.0, 150.0,-40.0

In [ ]:
%%time

dospi = 2*np.pi
lista = []
a = Model([(0,0), (1,0), (1,1), (0,1)])
b = a.move(2, -1/np.sqrt(2)/2, dospi/8)
c = a.birth(2, -0.5, dospi/4)
d = a.death(7)
e = Model_from_file('../TrueModel.csv')
# e[:,1] *= -1

modedict={
    'Igual':       a,
    'Simple':      b,
    'Creación':    c,
    'Destrucción': d,
    'TrueModel':   e
    }
for s, x in modedict.items():
  lista.append(x)
  print('//////////////////////////////////////////////////////////')
  print(s)
  print('----------------------------------------------------------')
  print('Model: ', x)
  print('Area: ', x.area())
  print('Center of mass: ', x.Cmass())
  print('----------------------------------------------------------')
  display_svg(Polygon(*map(Point, a)))
  display_svg(Polygon(*map(Point, x)))
  print('Vectors:\n',np.array([x.vectors(i)
                    for i in range(len(x))]))
  print('Angles:\n', np.array([np.degrees(x.angle(i))
                    for i in range(len(x))]))
  print('Dists:\n', np.array([list(x.dists(i)) 
                    for i in range(len(x))]))
  print('Mindists:\n', np.array([min(*x.dists(i))
                    for i in range(len(x))]))
  print('----------------------------------------------------------')
  print('DVector:\n',np.array([x.dvector(i)
                    for i in range(len(x))]))
  print('DRadius:', np.array([x.dradius(i) 
                    for i in range(len(x))]))
  print('BCateto:', np.array([x.bcat(i)
                    for i in range(len(x))]))
  print('DCateto:', np.array([x.dcat(i)
                    for i in range(len(x))]))
  print('----------------------------------------------------------')

In [ ]:
%%file ../TrueModel_from_GeoGebra.tsv
56.3963544480435	-54.2857104046646
106.082038697313	-31.1312167739369
154.802952378636	-34.9902990457249
196.770472084329	-52.3561692687706
207.382948331746	-82.7464421591004
181.816528281151	-96.7356153943317
163.485887490158	-125.196347148767
124.412679488306	-120.854879593006
81.4803892146655	-98.6651565302256
58.3258955839375	-69.2396542078427

In [ ]:
IFrame("https://www.geogebra.org/classic#spreadsheet", 1280, 720)

In [ ]:
%%time

sigma = 0.2
gamma = 1.6
Maxk  = 20
ang   = np.pi

old_settings = np.geterr()
new_settings = {
    'divide':  'raise',
    'invalid': 'raise',
    'over':    'ignore',
    'under':   'ignore'
    }
settlist = ['old_settings','new_settings']
display(pd.DataFrame(
    list(map(eval, settlist)),
    index=settlist
    ))
"""
ignore: Take no action when the exception occurs.
warn: Print a RuntimeWarning (via the Python warnings module).
raise: Raise a FloatingPointError.
call: Call a function specified using the seterrcall function.
print: Print a warning directly to stdout.
log: Record error in a Log object specified by seterrcall.
"""

TrueModel = Model_from_GeoGebra('../TrueModel_from_GeoGebra.tsv')
try:
  old_polygons = Model_from_file('old_polygons.csv')
  goodpolygons = Model_from_file('goodpolygons.csv')
  polygons     = Model_from_file('polygons.csv')
except:
  print('No hay archivos polygons\n')

w = np.ones(Maxk)
b = np.ones(Maxk)
d = np.ones(Maxk)
b[-1] = 0
d[2] = 0
ps = np.array([list(p/sum(p)) for p in np.stack((w,b,d)).T])
ps[0:2][:] = np.zeros((2,3))

XZ     = np.linspace(start=(0,0), stop=(500,0), num=25+1)
XZgrid = np.linspace(start=(0,0), stop=(500,0), num=1000+1)

XZgrid[:,1] = 5*(1.5+np.cos(.1*XZgrid[:,0]))
XZ[:,1] = interp1d(*XZgrid.T, kind='cubic')(XZ[:,0])

g = TrueModel.gravitational(XZ)
XZg = np.column_stack((XZ, g))
np.savetxt(
    '../Gravity.tsv',
    XZg,
    delimiter='\t',
    fmt='%.6g'
    )

XZg = np.genfromtxt('../Gravity.tsv')
XZ = XZg[:,:2]
g  = XZg[:,-1]

XZmin = XZ
del XZgrid
XZgrid = xz_iterp(
    XZ, 0, 500,
    num=1000+1,
    kind='cubic'
    )

InitModel = RegularModel(
    np.array([400, -100]),
    r=30,
    phi=-np.pi/4,
    n=3
    )
# InitModel = polygons[-1]
# %mv polygons.csv old_polygons.csv

print('InitModel Acceptance Test:')
display(acceptanceTest(InitModel, 0, XZ, g, ps))
print('\n')

try:
  print('Init Model:\n')
  display_svg(Polygon(*map(Point, InitModel.copy())))
  print('True Model:\n')
  display_svg(Polygon(*map(Point, TrueModel.copy())))
except:
  print('Error or Polygon has intersecting sides.')

Xi    = [XZgrid[:,0], InitModel.gravitational(XZgrid)]
Xg    = [XZgrid[:,0], TrueModel.gravitational(XZgrid)]
Xgmin = [XZmin[:,0],  TrueModel.gravitational(XZmin) ]

fig, ax_z = plt.subplots(figsize=np.array([16,9])*3/5)
ax_z.tick_params(
    top=True,
    labeltop=True
    )
ax_g = ax_z.twinx()
ax_z.axhline(c='k', lw=1, zorder=0)

ax_g.plot(
    *Xg,
    color='k'
    )
ax_g.scatter(
    *Xgmin,
    marker='.',
    color='k'
    )
lineg = Line2D(
  [0],[0],
  marker='.',
  color='k',
  label='Verdadera'
  )

ax_z.plot(
    *XZgrid.T,
    color='tab:green',
    label='_nolegend_'
)
ax_z.fill_between(
    *XZgrid.T,
    color='tab:green',
    alpha=3/5
)
ax_z.scatter(
    *XZmin.T,
    marker='.',
    color='tab:green',
    label='_nolegend_'
)
linez = Line2D(
  [0],[0],
  marker='.',
  color='tab:green',
  label='Terreno'
  )

Truepoly = patches.Polygon(
    TrueModel,
    closed=True,
    Fill=True,
    edgecolor=None,
    label='Verdadero'
    )
ax_z.add_patch(Truepoly)

ax_z.scatter(
    *TrueModel.Cmass(),
    marker='x',
    color='r',
    zorder=2
    )

ax_z.set_xlabel(r'$x$')
ax_z.set_ylabel(r'$z$', rotation=0)
ax_g.set_ylabel(r'$\quad\Delta{g}$', rotation=0, y=5/6)

ax_z.set_xlim(0, 500)
ax_z.set_ylim(ymin=-200)
ax_g.set_yticks(np.linspace(0, 100, 5))
align.yaxes(ax_z, 0, ax_g, 0, 2/3)

ax_g.legend(
    handles=[lineg, linez],
    title='Gravimetría',
    bbox_to_anchor=(0.25, -.1),
    ncol=3,
    loc='upper center'
    )
ax_z.legend(
    handles=[Truepoly],
    title='Modelo',
    bbox_to_anchor=(0.75, -.1),
    ncol=2,
    loc='upper center'
    )
plt.savefig('InitPlot0.pdf', bbox_inches='tight')
plt.show()

In [ ]:
!cat ../Gravity.tsv

In [ ]:
fig, ax_z = plt.subplots(figsize=np.array([16,9])*3/5)
ax_z.tick_params(
    top=True,
    labeltop=True
    )
ax_g = ax_z.twinx()
ax_z.axhline(c='k', lw=1, zorder=0)

lineg = ax_g.scatter(
    *Xgmin,
    marker='d',
    color='k',
    label='Verdadera'
    )

ax_g.plot(
    *Xi,
    linestyle=':',
    color='gray'
    )
linei = Line2D(
  [0],[0],
  linestyle=':',
  color='gray',
  label='Inicial'
  )

ax_z.plot(
    *XZgrid.T,
    color='tab:green',
    label='_nolegend_'
)
ax_z.fill_between(
    *XZgrid.T,
    color='tab:green',
    alpha=3/5
)
ax_z.scatter(
    *XZmin.T,
    marker='.',
    color='tab:green',
    label='_nolegend_'
)
linez = Line2D(
  [0],[0],
  marker='.',
  color='tab:green',
  label='Terreno'
  )
Initpoly = patches.Polygon(
    InitModel,
    closed=True,
    Fill=False,
    linestyle='--',
    edgecolor='k',
    label='Inicial'
    )
ax_z.add_patch(Initpoly)

ax_z.scatter(
    *InitModel.Cmass(),
    marker='.',
    color='k',
    zorder=2
    )

ax_z.set_xlabel(r'$x$')
ax_z.set_ylabel(r'$z$', rotation=0)
ax_g.set_ylabel(r'$\quad\Delta{g}$', rotation=0, y=5/6)

ax_z.set_xlim(0, 500)
ax_z.set_ylim(ymin=-200)
ax_g.set_yticks(np.linspace(0, 100, 5))
align.yaxes(ax_z, 0, ax_g, 0, 2/3)

ax_g.legend(
    handles=[lineg, linei, linez],
    title='Gravimetría',
    bbox_to_anchor=(0.33, -.1),
    ncol=3,
    loc='upper center'
    )
ax_z.legend(
    handles=[Initpoly],
    title='Modelo',
    bbox_to_anchor=(0.8, -.1),
    ncol=2,
    loc='upper center'
    )
plt.savefig('InitPlot1.pdf', bbox_inches='tight')
plt.show()

In [ ]:
%%time

maxsteps = 12*10**3
miniters = 10

display(pd.DataFrame.from_dict(
    new_settings,
    orient='index',
    columns=['treatment']
    ).T)
print('Steps:')
start=dt.now()
with open('polygons.csv', 'w') as polygons,\
     open('itertimes.csv', 'w') as itertimes,\
     open('moves.csv', 'w') as moves,\
     open('errit.csv', 'w') as errit:
  pbar=tqdm(
      total=maxsteps,
      miniters=miniters,
      )
  loops=1
  steps=1
  errors=0
  oldtime=0
  polygons.write(str(InitModel.copy()) + '\n')
  moves.write('p\n')
  errit.write(f'{loops:d}\n')
  initk = len(InitModel)
  oldModel = InitModel
  oldi = rg.randint(initk)
  pbar.update(1)
  sys.stdout.write(
      f'\rErrors = {errors:07d}'+
      f'\tErrors/Loops = {errors/loops:.6f}'
      )
  while steps<maxsteps:
    loops += 1
    with np.errstate(**new_settings):
      try:
        alpha, newModel, newi, move = acceptanceTest(
            oldModel, oldi,
            XZ, g, ps,
            sigma=sigma,
            gamma=gamma,
            ang=ang
            )
        # Metropolis Condition and Shamos-Huey algorithm
        if rg.random()<min(1,alpha) and newModel.notintersect(): 
          polygons.write(str(newModel.copy()) + '\n')
          oldModel = newModel
          oldi     = newi
          moves.write(move + '\n')
          if steps%100==0:
            ittime = (dt.now()-start).total_seconds()
            itertimes.write(f'{ittime-oldtime:f}\n')
            oldtime = ittime
          steps += 1
          pbar.update(1)
      except FloatingPointError:
        errors += 1
        sys.stdout.write(
            f'\rErrors = {errors:07d}'+\
            f'\tErrors/Loops = {errors/loops:.6f}'
            )
        sys.stdout.flush()
        errit.write(f'{loops:d}\n')
  errit.write(f'{loops:d}\n')
  stop = dt.now()
  duration = (stop-start).total_seconds()
  itertimes.write(f'{duration-oldtime:f}\n')
  durationstr = dt.fromtimestamp(duration).strftime('%H:%M:%S.%f')
  polygons.close()
  itertimes.close()
  moves.close()
  errit.close()
pbar.close()
reject = loops  - steps
rejerr = reject - errors
print(
    f'\rErrors = {errors:07d}',
    f'\tErrors/Loops = {errors/loops:.6f}',
    end='\n')
print(
    f' Steps = {steps:07d}',
    f'\t Steps/Loops = {steps/loops:.6f}',
    end='\n')
print(f' Loops = {loops:07d}')
print(f'Reject = {reject:07d}')
print(f'Rejerr = {rejerr:07d}')
print(f'Tiempo:  {durationstr}')        

In [ ]:
%%capture out

print('Parameters:')
print(f'sigma: {sigma:g}',
      f'gamma: {gamma:g}',
      f'  ang: {np.degrees(ang):g}',
      sep='\n'
)
print('\nExecution:')
print(
    f'\rErrors = {errors:07d}'+\
    f'\tErrors/Loops = {errors/loops:.6f}',
    end='\n')
print(
    f' Steps = {steps:07d}'+\
    f'\t Steps/Loops = {steps/loops:.6f}',
    end='\n')
print(f' Loops = {loops:07d}')
print(f'Reject = {reject:07d}')
print(f'Rejerr = {rejerr:07d}')
print(f'Tiempo:  {durationstr}') 
print('\nNumba Config:')
!cat .numba_config.yaml

In [ ]:
with open('out.txt', 'w') as f:
  print(out.stdout)
  f.write(out.stdout)

In [ ]:
%%time

dfmoves = pd.read_csv(
    'moves.csv',
    header=None,
    names=['moves']
    )
movescounter = Counter(dfmoves['moves'])
new_keys = ['Simple','Creación','Destrucción']
for oldkey, newkey in zip(movescounter.keys(), new_keys):
  movescounter[newkey] = movescounter.pop(oldkey)
dfmovesfreq = pd.DataFrame.from_dict(
    movescounter,
    orient='index',
    columns=['Freq']
    ).sort_index()
display(dfmovesfreq)
dfmovesfreq.plot.pie(y='Freq', autopct='%1.1f%%', legend=False)
plt.title('Freciencia en Tipos de Movimientos')
plt.savefig('movesfreq.pdf', bbox_inches='tight')
plt.ylabel(None)
plt.show()

In [ ]:
%%time

erritfi = np.genfromtxt('errit.csv', delimiter='\n')
fig, ax = plt.subplots(1)
ax.bar(
    erritfi,
    np.ones_like(erritfi),
    width=10,
    color='k',
    alpha=.75
    )
ax.get_yaxis().set_visible(False)
#ax.set_rasterized(True)
ax.set_xlim(0, erritfi[-1])
ax.set_ylim(0, 1)
ax.set_xlabel('Iteraciones')
plt.title('Errores numéricos durante las iteraciones')
plt.savefig('errit.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
%%time

dfitertimes = pd.read_csv(
    'itertimes.csv',
    header=None,
    names=['itertimes']
    )
fig, ax = plt.subplots(1)
ax.plot(
    dfitertimes, #.rolling(100).mean(),
    color='k'
    )
ax.tick_params(
    left=True,
    labelleft=True,
    right=True,
    labelright=True    
    )
ax.set_xlabel('Iteraciones')
ax.set_ylabel(r'$\Delta{t}$ (s)')
plt.title('Tiempo cada 100 Iteraciones')
plt.savefig('itertimes.pdf', bbox_inches='tight')
plt.show()

In [ ]:
%%time

polygons = Model_from_file('polygons.csv')
lenp = len(polygons)
less = 1000
lesspolygons = [polygons[i] for i in range(0, lenp, int(lenp/less))]
InitModel = lesspolygons[0]
TestModel = lesspolygons[-1]
print('bigwalk:')
bigwalk = np.array([p.Cmass() for p in tqdm(polygons)])
print('biggeom:')
biggeom = np.array([p.Cgeom() for p in tqdm(polygons)])
print('walk:')
walk    = np.array([p.Cmass() for p in tqdm(lesspolygons)])
print('geom:')
geom    = np.array([p.Cgeom() for p in tqdm(lesspolygons)])

lenwalk  = len(walk)
points   = bigwalk.reshape(-1, 1, 2)
segments = np.concatenate(
    [points[:-1],
     points[1:]],
     axis=1)

lc = LineCollection(
    segments,
    cmap=plt.get_cmap('jet'),
    norm=plt.Normalize(0,lenwalk),
    zorder=2
    )
lc.set_array(np.arange(lenwalk))
lc.set_linewidth(2)
#lc.set_rasterized(True)

fig, ax = plt.subplots(1)
ax.scatter(
    *InitModel.Cmass(),
    marker='o',
    color='k',
    zorder=2
    )
ax.plot(
    *bigwalk.T,
    color='k',
    alpha=0,
    zorder=0,
    )
ax.plot(
    *biggeom.T,
    color='k',
    alpha=0.2,
    zorder=0,
    )
ax.scatter(
    *TrueModel.Cmass(),
    marker='x',
    color='r',
    zorder=2
    )
ax.scatter(
    *TrueModel.Cgeom(),
    marker='x',
    color='gray',
    zorder=2
    )
ax.tick_params(
    bottom=True,
    labelbottom=True,
    top=True,
    labeltop=True,
    left=True,
    labelleft=True,
    right=True,
    labelright=True    
    )
ax.add_collection(lc)
ax.set_xlabel('x')
ax.set_ylabel('z', rotation=0)
plt.title('Recorrido de Caminos')
plt.savefig('walk.pdf', bbox_inches='tight')
plt.show()

In [ ]:
%%time

print('kaes:')
kaes = [len(TestModel) for TestModel in tqdm(polygons)]

plt.bar(*zip(*Counter(kaes).items()))
plt.xticks(np.arange(3, Maxk+1))
plt.yscale('log')
plt.title('Histograma Logarítmico para Numero de Vertices')
plt.xlabel('Numero de Vertices')
plt.ylabel('Frecuencia')
plt.savefig('hist.pdf', bbox_inches='tight')
plt.show()

In [ ]:
%%time

koc = np.delete(
        np.dstack(
          (kaes, np.roll(kaes, -1))),
          -1, axis=1
          )
df = pd.DataFrame(
    *koc,
    columns=['open','close']
    )
df['low']  = np.fromiter(map(np.amin, *koc), dtype=np.int)
df['high'] = np.fromiter(map(np.amax, *koc), dtype=np.int)
df.to_csv('kaes.csv')

fig = go.Figure(
    data=[go.Candlestick(
      x=df.index,
      open=df.open,
      high=df.high,
      low=df.low,
      close=df.close
    )])

fig.update_layout(
    template='plotly_white',
    margin=go.layout.Margin(
        l=1, #left margin
        r=1, #right margin
        b=1, #bottom margin
        t=26 #top margin
    ),
    width=800,
    height=450,
    title='Evolución en Numero de Vertices',
    title_x=0.5,
    xaxis_title='Iteraciones',
    yaxis_title='Numero de Vertices',
    yaxis=dict(
        range=[3, Maxk],
        tickmode='linear',
        tick0=3,
        dtick=1
        ),
    xaxis_rangeslider_visible=False,
    )
fig.write_image('kaes.pdf')
fig.show()

In [ ]:
%%time

print('likehood:')
likehood = [Likelihood(TestModel, XZ, g, sigma=sigma)\
  for TestModel in tqdm(polygons)]
print('priorden:')
priorden = [Prior_density(TestModel, gamma=gamma)\
  for TestModel in tqdm(polygons)]
   
try:
  with np.errstate(all='warn'):
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    # https://stackoverflow.com/a/3513150
    # ax1.set_yscale('symlog')
    ax1.set_yscale('log', nonposy='mask')
    # ax2.set_yscale('symlog')
    ax2.set_yscale('log', nonposy='mask')
    c1='darkblue'
    c2='darkred'
    ax1.plot(
        priorden,
        color=c1
        )
    ax2.plot(
        likehood,
        color=c2
        )
    ax1.tick_params(
        axis='y',
        colors=c1
        )
    ax2.tick_params(
        axis='y',
        colors=c2
        )
    ax1.set_xlabel(
        'Iteraciones'
        )
    ax1.set_ylabel(
        'Prior density',
        color=c1
        )
    ax2.set_ylabel(
        'Likelihood',
        color=c2
        )
    plt.title('Evolución de Funciones')
    plt.savefig('evolution.pdf', bbox_inches='tight')
    plt.show()
except:
  print('Error')

In [ ]:
%%time

fig, ax_z = plt.subplots(figsize=np.array([16,9])*3/5)
ax_z.tick_params(
    top=True,
    labeltop=True
    )
ax_g = ax_z.twinx()
ax_z.axhline(c='k', lw=1, zorder=0)

Xy    = [XZgrid[:,0], lesspolygons[-1].gravitational(XZgrid)]
Xymin = [XZmin[:,0],  lesspolygons[-1].gravitational(XZmin) ]

ax_g.plot(
    *Xg,
    color='k'
    )
ax_g.scatter(
    *Xgmin,
    marker='.',
    color='k'
    )
lineg = Line2D(
  [0],[0],
  marker='.',
  color='k',
  label='Verdadera'
  )

ax_g.plot(
    *Xi,
    linestyle=':',
    color='gray'
    )
linei = Line2D(
  [0],[0],
  linestyle=':',
  color='gray',
  label='Inicial'
  )

ax_g.plot(
    *Xy,
    color='gray'
    )
ax_g.scatter(
    *Xymin,
    marker='.',
    color='gray',
    label='_nolegend_'
    )
liney = Line2D(
  [0],[0],
  marker='.',
  color='gray',
  label='Prueba'
  )

ax_z.plot(
    *XZgrid.T,
    color='tab:green',
    label='_nolegend_'
)
ax_z.fill_between(
    *XZgrid.T,
    color='tab:green',
    alpha=3/5
)
ax_z.scatter(
    *XZmin.T,
    marker='.',
    color='tab:green',
    label='_nolegend_'
)
linez = Line2D(
  [0],[0],
  marker='.',
  color='tab:green',
  label='Terreno'
  )

Truepoly = patches.Polygon(
    TrueModel,
    closed=True,
    Fill=True,
    edgecolor=None,
    label='Verdadero'
    )
ax_z.add_patch(Truepoly)
Initpoly = patches.Polygon(
    lesspolygons[0],
    closed=True,
    Fill=False,
    linestyle='--',
    edgecolor='k',
    label='Inicial'
    )
ax_z.add_patch(Initpoly)

Testpoly = patches.Polygon(
    lesspolygons[-1],
    closed=True,
    Fill=False,
    edgecolor='k',
    label='Prueba'
    )
ax_z.add_patch(Testpoly)

ax_z.scatter(
    *lesspolygons[0].Cmass(),
    marker='.',
    color='k',
    zorder=2
    )
ax_z.scatter(
    *TrueModel.Cmass(),
    marker='x',
    color='r',
    zorder=2
    )
ax_z.plot(
    *walk.T,
    linewidth=0.5,
    color='k',
    )

ax_z.set_xlabel(r'$x$')
ax_z.set_ylabel(r'$z$', rotation=0)
ax_g.set_ylabel(r'$\quad\Delta{g}$', rotation=0, y=5/6)

ax_z.set_xlim(0, 500)
ax_z.set_ylim(ymin=-200)
ax_g.set_yticks(np.linspace(0, 100, 5))
align.yaxes(ax_z, 0, ax_g, 0, 2/3)


ax_g.legend(
    handles=[lineg, linei, liney, linez],
    title='Gravimetría',
    bbox_to_anchor=(.4, -.1),
    ncol=2,
    loc='upper right'
    )
ax_z.legend(
    handles=[Truepoly, Initpoly, Testpoly],
    title='Modelo',
    bbox_to_anchor=(.6, -.1),
    ncol=2,
    loc='upper left'
    )
plt.savefig('Plot0.pdf', bbox_inches='tight')
plt.show()

In [ ]:
%%time

fig, ax_z = plt.subplots(figsize=np.array([16,9])*3/5)
ax_z.tick_params(
    top=True,
    labeltop=True
    )
ax_g = ax_z.twinx()
ax_z.axhline(c='k', lw=1, zorder=0)

Xy    = [XZgrid[:,0], lesspolygons[-1].gravitational(XZgrid)]
Xymin = [XZmin[:,0],  lesspolygons[-1].gravitational(XZmin) ]

lineg = ax_g.scatter(
    *Xgmin,
    marker='d',
    color='k',
    label='Verdadera'
    )

ax_g.plot(
    *Xi,
    linestyle=':',
    color='gray'
    )
linei = Line2D(
  [0],[0],
  linestyle=':',
  color='gray',
  label='Inicial'
  )

ax_g.plot(
    *Xy,
    color='gray'
    )
ax_g.scatter(
    *Xymin,
    marker='.',
    color='gray',
    label='_nolegend_'
    )
liney = Line2D(
  [0],[0],
  marker='.',
  color='gray',
  label='Prueba'
  )

ax_z.plot(
    *XZgrid.T,
    color='tab:green',
    label='_nolegend_'
)
ax_z.fill_between(
    *XZgrid.T,
    color='tab:green',
    alpha=3/5
)
ax_z.scatter(
    *XZmin.T,
    marker='.',
    color='tab:green',
    label='_nolegend_'
)
linez = Line2D(
  [0],[0],
  marker='.',
  color='tab:green',
  label='Terreno'
  )

Initpoly = patches.Polygon(
    lesspolygons[0],
    closed=True,
    Fill=False,
    linestyle='--',
    edgecolor='k',
    label='Inicial'
    )
ax_z.add_patch(Initpoly)

Testpoly = patches.Polygon(
    lesspolygons[-1],
    closed=True,
    Fill=False,
    edgecolor='k',
    label='Prueba'
    )
ax_z.add_patch(Testpoly)

ax_z.scatter(
    *lesspolygons[0].Cmass(),
    marker='.',
    color='k',
    zorder=2
    )
ax_z.plot(
    *walk.T,
    linewidth=0.5,
    color='k',
    )

ax_z.set_xlabel(r'$x$')
ax_z.set_ylabel(r'$z$', rotation=0)
ax_g.set_ylabel(r'$\quad\Delta{g}$', rotation=0, y=5/6)

ax_z.set_xlim(0, 500)
ax_z.set_ylim(ymin=-200)
ax_g.set_yticks(np.linspace(0, 100, 5))
align.yaxes(ax_z, 0, ax_g, 0, 2/3)


ax_g.legend(
    handles=[lineg, linei, liney, linez],
    title='Gravimetría',
    bbox_to_anchor=(.4, -.1),
    ncol=2,
    loc='upper right'
    )
ax_z.legend(
    handles=[Initpoly, Testpoly],
    title='Modelo',
    bbox_to_anchor=(.6, -.1),
    ncol=2,
    loc='upper left'
    )
plt.savefig('Plot1.pdf', bbox_inches='tight')
plt.show()

In [ ]:
%%time
fps = 30
print(f'Graficando cada fotograma, puede tardar un tiempo...')
fig, ax_z = plt.subplots(figsize=np.array([16,9])*3/5, dpi=300)
ax_z.tick_params(
    top=True,
    labeltop=True
    )
ax_g = ax_z.twinx()
camera=Camera(fig)
ax_z.tick_params(
    top=True,
    labeltop=True
    )
for i, TestModel in tqdm(
    enumerate(lesspolygons),
    total=len(lesspolygons),
    miniters=10
    ):
  ax_z.axhline(c='k', lw=1, zorder=0)
  
  Xy    = [XZgrid[:,0], TestModel.gravitational(XZgrid)]
  Xymin = [XZmin[:,0],  TestModel.gravitational(XZmin) ]

  ax_g.plot(
      *Xg,
      color='k'
      )
  ax_g.scatter(
      *Xgmin,
      marker='.',
      color='k'
      )
  lineg = Line2D(
    [0],[0],
    marker='.',
    color='k',
    label='Verdadera'
    )

  ax_g.plot(
      *Xi,
      linestyle=':',
      color='gray'
      )
  linei = Line2D(
    [0],[0],
    linestyle=':',
    color='gray',
    label='Inicial'
    )

  ax_g.plot(
      *Xy,
      color='gray'
      )
  ax_g.scatter(
      *Xymin,
      marker='.',
      color='gray',
      label='_nolegend_'
      )
  liney = Line2D(
    [0],[0],
    marker='.',
    color='gray',
    label='Prueba'
    )

  ax_z.plot(
      *XZgrid.T,
      color='tab:green',
      label='_nolegend_'
  )
  ax_z.fill_between(
      *XZgrid.T,
      color='tab:green',
      alpha=3/5
  )
  ax_z.scatter(
      *XZmin.T,
      marker='.',
      color='tab:green',
      label='_nolegend_'
  )
  linez = Line2D(
    [0],[0],
    marker='.',
    color='tab:green',
    label='Terreno'
    )

  Truepoly = patches.Polygon(
      TrueModel,
      closed=True,
      Fill=True,
      edgecolor=None,
      label='Verdadero'
      )
  ax_z.add_patch(Truepoly)
  Initpoly = patches.Polygon(
      lesspolygons[0],
      closed=True,
      Fill=False,
      linestyle='--',
      edgecolor='k',
      label='Inicial'
      )
  ax_z.add_patch(Initpoly)

  Testpoly = patches.Polygon(
      TestModel,
      closed=True,
      Fill=False,
      edgecolor='k',
      label='Prueba'
      )
  ax_z.add_patch(Testpoly)

  ax_z.scatter(
      *lesspolygons[0].Cmass(),
      marker='.',
      color='k',
      zorder=2
      )
  ax_z.scatter(
      *TrueModel.Cmass(),
      marker='x',
      color='r',
      zorder=2
      )
  ax_z.plot(
      *walk[:i].T,
      linewidth=0.5,
      color='k',
      )

  ax_z.set_xlabel(r'$x$')
  ax_z.set_ylabel(r'$z$', rotation=0)
  ax_g.set_ylabel(r'$\quad\Delta{g}$', rotation=0, y=5/6)

  ax_z.set_xlim(0, 500)
  ax_z.set_ylim(ymin=-200)
  ax_g.set_yticks(np.linspace(0, 100, 5))
  align.yaxes(ax_z, 0, ax_g, 0, 2/3)

  ax_g.legend(
      handles=[lineg, linei, liney, linez],
      title='Gravimetría',
      bbox_to_anchor=(.4, -.1),
      ncol=2,
      loc='upper right'
      )
  ax_z.legend(
      handles=[Truepoly, Initpoly, Testpoly],
      title='Modelo',
      bbox_to_anchor=(.6, -.1),
      ncol=2,
      loc='upper left'
      )
  plt.tight_layout(w_pad=0, h_pad=1)
  camera.snap()
plt.close()
animation=camera.animate(interval=less/fps, repeat=False)
#interval: Delay between frames in milliseconds. Defaults to 200.
filename='video0.mp4'
start=dt.now()
print(f'Guardando el archivo {filename}, puede tardar un tiempo...')
pbar=tqdm(total=less)
animation.save(filename, progress_callback=\
               lambda i, n: pbar.update(1)
               )
pbar.close()
stop=dt.now()
videoduration=int((stop-start).total_seconds())
videodurationstr=dt.fromtimestamp(videoduration).strftime('%M:%S')
print(f'Archivo {filename} Guardado!\n Tiempo: {videodurationstr}')
showvideo(filename)

In [ ]:
%%time

fps = 30
print(f'Graficando cada fotograma, puede tardar un tiempo...')
fig, ax_z = plt.subplots(figsize=np.array([16,9])*3/5, dpi=300)
ax_z.tick_params(
    top=True,
    labeltop=True
    )
ax_g = ax_z.twinx()
camera=Camera(fig)
ax_z.tick_params(
    top=True,
    labeltop=True
    )
for i, TestModel in tqdm(
    enumerate(lesspolygons),
    total=len(lesspolygons),
    miniters=10
    ):
  ax_z.axhline(c='k', lw=1, zorder=0)
  
  Xy    = [XZgrid[:,0], TestModel.gravitational(XZgrid)]
  Xymin = [XZmin[:,0],  TestModel.gravitational(XZmin) ]

  lineg = ax_g.scatter(
      *Xgmin,
      marker='d',
      color='k',
      label='Verdadera'
      )

  ax_g.plot(
      *Xi,
      linestyle=':',
      color='gray'
      )
  linei = Line2D(
    [0],[0],
    linestyle=':',
    color='gray',
    label='Inicial'
    )

  ax_g.plot(
      *Xy,
      color='gray'
      )
  ax_g.scatter(
      *Xymin,
      marker='.',
      color='gray',
      label='_nolegend_'
      )
  liney = Line2D(
    [0],[0],
    marker='.',
    color='gray',
    label='Prueba'
    )

  ax_z.plot(
      *XZgrid.T,
      color='tab:green',
      label='_nolegend_'
  )
  ax_z.fill_between(
      *XZgrid.T,
      color='tab:green',
      alpha=3/5
  )
  ax_z.scatter(
      *XZmin.T,
      marker='.',
      color='tab:green',
      label='_nolegend_'
  )
  linez = Line2D(
    [0],[0],
    marker='.',
    color='tab:green',
    label='Terreno'
    )

  Initpoly = patches.Polygon(
      lesspolygons[0],
      closed=True,
      Fill=False,
      linestyle='--',
      edgecolor='k',
      label='Inicial'
      )
  ax_z.add_patch(Initpoly)

  Testpoly = patches.Polygon(
      TestModel,
      closed=True,
      Fill=False,
      edgecolor='k',
      label='Prueba'
      )
  ax_z.add_patch(Testpoly)

  ax_z.scatter(
      *lesspolygons[0].Cmass(),
      marker='.',
      color='k',
      zorder=2
      )
  ax_z.plot(
      *walk[:i].T,
      linewidth=0.5,
      color='k',
      )

  ax_z.set_xlabel(r'$x$')
  ax_z.set_ylabel(r'$z$', rotation=0)
  ax_g.set_ylabel(r'$\quad\Delta{g}$', rotation=0, y=5/6)

  ax_z.set_xlim(0, 500)
  ax_z.set_ylim(ymin=-200)
  ax_g.set_yticks(np.linspace(0, 100, 5))
  align.yaxes(ax_z, 0, ax_g, 0, 2/3)

  ax_g.legend(
      handles=[lineg, linei, liney, linez],
      title='Gravimetría',
      bbox_to_anchor=(.4, -.1),
      ncol=2,
      loc='upper right'
      )
  ax_z.legend(
      handles=[Initpoly, Testpoly],
      title='Modelo',
      bbox_to_anchor=(.6, -.1),
      ncol=2,
      loc='upper left'
      )
  plt.tight_layout(w_pad=0, h_pad=1)
  camera.snap()
plt.close()
animation=camera.animate(interval=less/fps, repeat=False)
#interval: Delay between frames in milliseconds. Defaults to 200.
filename='video1.mp4'
start=dt.now()
print(f'Guardando el archivo {filename}, puede tardar un tiempo...')
pbar=tqdm(total=less)
animation.save(filename, progress_callback=\
               lambda i, n: pbar.update(1)
               )
pbar.close()
stop=dt.now()
videoduration=int((stop-start).total_seconds())
videodurationstr=dt.fromtimestamp(videoduration).strftime('%M:%S')
print(f'Archivo {filename} Guardado!\n Tiempo: {videodurationstr}')
showvideo(filename)

In [ ]:
%%time

#https://stackoverflow.com/a/4617069
try:
  with open('ok0_polygons.csv', 'r') as ok0,\
       open('ok1_polygons.csv', 'r') as ok1,\
       open('goodpolygons.csv', 'w') as good:
    p0 = ok0.readlines()
    p1 = ok1.readlines()
    lines = p0[:] + p1[1:]
    for line in lines:
      good.write(line)
    ok0.close()
    ok1.close()
    good.close()
except:
  print('No hay archivos?')

In [ ]:
%cd "{dir0}"
!rm -f zfiles.zip
!7z a -bso0 {name}.zip ./{name}/* ./{name}/.numba_config.yaml
!7z l {name}.zip
%cd "{workdir}"

In [ ]:
%ls -altrh